In [1]:
import gurobipy as gp
import numpy as np
import math
from numpy import random

In [2]:
def SLMFNE(size_of_action_space):
    m = size_of_action_space
    max_val = 100
    U1 = random.randint(max_val, size=(m,m,m))
    U2 = random.randint(max_val, size=(m,m,m))
    U3 = random.randint(max_val, size=(m,m,m))

    #Defining Dictionaries for various variables
    y13 = {}
    y23 = {}
    z = {}
    x1 = {}
    x2 = {}
    x3 = {}
    s1 = {}
    s2 = {}
    
    
    M = gp.Model()

    #Adding all the variables

    for i in range(m):
        s1[i] = M.addVar(vtype = gp.GRB.BINARY, name = "s1"+str(i))
        s2[i] = M.addVar(vtype = gp.GRB.BINARY, name = "s2"+str(i))
        x1[i] = M.addVar(lb = 0, vtype = gp.GRB.CONTINUOUS, name = "x1"+str(i))
        for j in range(m):
            y13[i,j] = M.addVar(lb = 0, ub = 1, vtype = gp.GRB.CONTINUOUS, name = "y13"+str(i)+'_'+str(j))
            x2[j] = M.addVar(lb = 0, vtype = gp.GRB.CONTINUOUS, name = "x2"+str(j))
            for k in range(m):
                y23[j,k] = M.addVar(lb = 0, ub = 1, vtype = gp.GRB.CONTINUOUS, name = "y23"+str(j)+'_'+str(k))
                z[i,j,k] = M.addVar(lb = 0, ub = 1, vtype = gp.GRB.CONTINUOUS, name = "z"+str(i)+'_'+str(j)+'_'+str(k))
                x3[k] = M.addVar(lb = 0, vtype = gp.GRB.CONTINUOUS, name = "x3"+str(k))
            
    v1 = M.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype = gp.GRB.CONTINUOUS)
    v2 = M.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype = gp.GRB.CONTINUOUS)

    BigM = 10000000000
    
    #Objective Function

    M.setObjective(gp.quicksum(U3[i,j,k]*z[i,j,k] for i in range(m) for j in range(m) for k in range(m)), sense = gp.GRB.MAXIMIZE)

    #Constraints

    M.addConstr(gp.quicksum(y13[i,k] for i in range(m) for j in range(m))==1)
    M.addConstr(gp.quicksum(y23[j,k] for j in range(m) for k in range(m))==1)
    M.addConstr(gp.quicksum(z[i,j,k] for i in range(m) for j in range(m) for k in range(m))==1)

    M.addConstr(gp.quicksum(x1[i] for i in range(m))==1)
    M.addConstr(gp.quicksum(x2[j] for j in range(m))==1)
    M.addConstr(gp.quicksum(x3[k] for k in range(m))==1)

    for i in range(m):
        M.addConstr(v1 - gp.quicksum(U1[i,j,k]*y23[j,k] for j in range(m) for k in range(m))<=BigM*s1[i])
        M.addConstr(x1[i]<=1-s1[i])
        M.addConstr(v1 >= gp.quicksum(U1[i,j,k]*y23[j,k] for j in range(m) for k in range(m)))
        for k in range(m):
            M.addConstr(y13[i,k]==x1[i]*x3[k])
        
    for j in range(m):
        M.addConstr(v2 - gp.quicksum(U2[i,j,k]*y13[i,k] for i in range(m) for k in range(m))<=BigM*s1[i])
        M.addConstr(x2[j]<=1-s2[j])
        M.addConstr(v2 >= gp.quicksum(U2[i,j,k]*y13[i,k] for i in range(m) for k in range(m)))
        for k in range(m):
            M.addConstr(y23[j,k]==x2[j]*x3[k])

    M.addConstrs(z[i,j,k]==x1[i]*y23[j,k] for i in range(m) for j in range(m) for k in range(m))
    M.update()
    
    M.params.NonConvex = 2
    M.Params.OutputFlag = 0
    M.optimize()
    print("(Optimal value, Runtime, Optimality Gap)", M.objVal, M.Runtime, M.MIPGap)

In [ ]:
i = 4

while(i!=13):
    SLMFNE(i)
    i = i+1

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
(Optimal value, Runtime, Optimality Gap) 85.69350961538461 0.1884899139404297 0.0
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
(Optimal value, Runtime, Optimality Gap) 83.81388888888894 1.4721870422363281 0.0
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
(Optimal value, Runtime, Optimality Gap) 88.83333774364394 1.7703533172607422 2.5672601835120285e-06
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
(Optimal value, Runtime, Optimality Gap) 95.42857142857143 0.8614482879638672 0.0
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
(Optimal value, Runtime, Optimality Gap) 97.0 0.9811458587646484 0.0
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
(Optimal value, Runtime, Optimality Gap) 96.66666666666671 6.199146270751953 0.0

In [3]:
SLMFNE(12)

Academic license - for non-commercial use only - expires 2022-10-15
Using license file C:\Users\Dev\gurobi.lic
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
(Optimal value, Runtime, Optimality Gap) 98.33333333333334 13.523269653320312 0.0
